# Προχωρημένα Θέματα Βάσεων Δεδομένων

**Ονοματεπώνυμο:** Κωνσταντίνος Διβριώτης

**ΑΜ:** 03114140

## Query 5: 

Nα υπολογιστεί, ανά αστυνομικό τμήμα, ο αριθμός εγκλημάτων που έλαβαν χώρα πλησιέστερα σε αυτό, καθώς και η μέση απόστασή του από τις τοποθεσίες όπου σημειώθηκαν τα συγκεκριμένα περιστατικά. 

Τα αποτελέσματα να εμφανιστούν ταξινομημένα κατά αριθμό περιστατικών, με φθίνουσα σειρά.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PoliceStationDistanceAnalysis") \
    .getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1049,application_1732639283265_1016,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Χρησιμοποιούμε 8 cores και 16GB μνήμης με τα ακόλουθα configurations:
- 2 executors x 4 cores  / 8GB memory
- 4 executors x 2 cores  / 4GB memory
- 8 executors x 1 core  / 2GB memory

Εκτελούμε κάθε φορά το αντίστοιχο κελί για να επιλέξουμε το κατάλληλο configuration:

#### 2 executors x 4 cores / 8GB memory

In [3]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "8g",
        "spark.executor.cores": "4"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1051,application_1732639283265_1018,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
983,application_1732639283265_0950,pyspark,idle,Link,Link,None,
984,application_1732639283265_0951,pyspark,idle,Link,Link,None,
986,application_1732639283265_0953,pyspark,idle,Link,Link,None,
993,application_1732639283265_0960,pyspark,idle,Link,Link,None,
996,application_1732639283265_0963,pyspark,idle,Link,Link,None,
999,application_1732639283265_0966,pyspark,idle,Link,Link,None,
1001,application_1732639283265_0968,pyspark,idle,Link,Link,None,
1003,application_1732639283265_0970,pyspark,idle,Link,Link,None,
1004,application_1732639283265_0971,pyspark,idle,Link,Link,None,
1010,application_1732639283265_0977,pyspark,idle,Link,Link,None,


#### 4 executors x 2 cores / 4GB memory

In [11]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "4g",
        "spark.executor.cores": "2"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1052,application_1732639283265_1019,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
983,application_1732639283265_0950,pyspark,idle,Link,Link,None,
984,application_1732639283265_0951,pyspark,idle,Link,Link,None,
986,application_1732639283265_0953,pyspark,idle,Link,Link,None,
993,application_1732639283265_0960,pyspark,idle,Link,Link,None,
996,application_1732639283265_0963,pyspark,idle,Link,Link,None,
999,application_1732639283265_0966,pyspark,idle,Link,Link,None,
1001,application_1732639283265_0968,pyspark,idle,Link,Link,None,
1003,application_1732639283265_0970,pyspark,idle,Link,Link,None,
1004,application_1732639283265_0971,pyspark,idle,Link,Link,None,
1010,application_1732639283265_0977,pyspark,idle,Link,Link,None,


#### 8 executors x 1 core / 8GB memory

In [19]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "8",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1053,application_1732639283265_1020,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
983,application_1732639283265_0950,pyspark,idle,Link,Link,None,
984,application_1732639283265_0951,pyspark,idle,Link,Link,None,
986,application_1732639283265_0953,pyspark,idle,Link,Link,None,
993,application_1732639283265_0960,pyspark,idle,Link,Link,None,
996,application_1732639283265_0963,pyspark,idle,Link,Link,None,
999,application_1732639283265_0966,pyspark,idle,Link,Link,None,
1001,application_1732639283265_0968,pyspark,idle,Link,Link,None,
1003,application_1732639283265_0970,pyspark,idle,Link,Link,None,
1004,application_1732639283265_0971,pyspark,idle,Link,Link,None,
1010,application_1732639283265_0977,pyspark,idle,Link,Link,None,


In [20]:
conf = spark.sparkContext.getConf()
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Cores:", conf.get("spark.executor.cores"))
print("Executor Memory:", conf.get("spark.executor.memory"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: 8
Executor Cores: 1
Executor Memory: 2g

In [21]:
from sedona.spark import *

sedona = SedonaContext.create(spark)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
DATA_BUCKET = "s3://initial-notebook-data-bucket-dblab-905418150721"
GROUP_BUCKET = "s3://groups-bucket-dblab-905418150721/group15"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Διάβασμα αρχείων εισόδου

In [23]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType

police_stations_schema = StructType([
    StructField("X", DoubleType()),
    StructField("Y", DoubleType()),
    StructField("FID", IntegerType()),
    StructField("DIVISION", StringType()),
    StructField("LOCATION", StringType()),
    StructField("PREC", IntegerType())
])

# Μετατρέπουμε τις στήλες X, Y σε geometry με το ST_POINT
police_stations = spark.read.csv(f"{DATA_BUCKET}/LA_Police_Stations.csv",
                                 header=True, \
                                 schema=police_stations_schema
                                ) \
                        .withColumn("geom", ST_Point("X", "Y")) \
                        .select("geom", "DIVISION")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col

# Ορισμός του schema των dataset
crimes_schema = StructType([
    StructField("DR_NO", StringType()),
    StructField("Date Rptd", StringType()),
    StructField("DATE OCC", StringType()),
    StructField("TIME OCC", StringType()),
    StructField("AREA", IntegerType()),
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", StringType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType()),
    StructField("Premis Cd", StringType()),
    StructField("Premis Desc", StringType()),
    StructField("Weapon Used Cd", IntegerType()),
    StructField("Weapon Desc", StringType()),
    StructField("Status", StringType()),
    StructField("Status Desc", StringType()),
    StructField("Crm Cd 1", IntegerType()),
    StructField("Crm Cd 2", IntegerType()),
    StructField("Crm Cd 3", IntegerType()),
    StructField("Crm Cd 4", IntegerType()),
    StructField("LOCATION", StringType()),
    StructField("Cross Street", StringType()),
    StructField("LAT", DoubleType()),
    StructField("LON", DoubleType())
])

# Διαβάζουμε τα 2 datasets (2010-2019 και 2020-σήμερα) και τα συνενώνουμε σε 1
crime_data_2010_2019 = spark.read.csv(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, schema=crimes_schema)
crime_data_2020_present = spark.read.csv(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, schema=crimes_schema)
crime_data = crime_data_2010_2019.union(crime_data_2020_present)

# Μετατρέπουμε τις στήλες LAT, LON σε geometry με το ST_POINT και
# φιλτράρουμε τα δεδομένα που δεν αφορούν το Null Island (0,0)
crime_data = crime_data \
                .withColumn("geom", ST_Point("LON", "LAT")) \
                .filter((col("geom") != ST_Point(0, 0))) \
                .select("DR_NO", "geom")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Υλοποίηση με DataFrame

In [27]:
from pyspark.sql.functions import col, rank, avg, count
from pyspark.sql import Window
import time

start_time = time.time()

# Cross Join των crime_data, police_stations και υπολογισμός απόστασης
crime_ps_dist = crime_data \
                    .crossJoin(police_stations) \
                    .withColumn("Distance", ST_DistanceSphere(
                            crime_data["geom"], 
                            police_stations["geom"]
                        ) / 1000  # divide with 1000 to converτ into km
                    )

# Εύρεση του τμήματος με τη μικρότερη απόσταση από κάθε έγκλημα
window_spec = Window.partitionBy("DR_NO").orderBy(col("Distance").asc())
ranked_dist = crime_ps_dist.withColumn("rank", rank().over(window_spec)) \
                            .filter(col("rank") == 1)

# Υπολογισμός της μέσης απόστασης κάθε αστυνομικού τμήματος από
# τα περιστατικά, καθώς και του πλήθους των περιστατικών
result = ranked_dist \
                    .groupBy("DIVISION") \
                    .agg(
                        avg("Distance").alias("avg_distance"),
                        count("*").alias("#")
                    ) \
                    .withColumnRenamed("DIVISION", "division") \
                    .orderBy("#", ascending=False)

result.show(n=21)

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+------+
|        division|      avg_distance|     #|
+----------------+------------------+------+
|       HOLLYWOOD|2.0762639601787183|224340|
|        VAN NUYS|2.9533697428197794|210134|
|       SOUTHWEST| 2.191398805780884|188901|
|        WILSHIRE| 2.592665532978765|185996|
|     77TH STREET|1.7165449719700956|171827|
|         OLYMPIC| 1.723603697178096|170897|
| NORTH HOLLYWOOD|2.6430060941415645|167854|
|         PACIFIC| 3.850070655307917|161359|
|         CENTRAL|0.9924764374568906|153871|
|         RAMPART| 1.534534187919012|152736|
|       SOUTHEAST|  2.42186621588818|152176|
|     WEST VALLEY|  3.03567121631407|138643|
|         TOPANGA|3.2969548417555528|138217|
|        FOOTHILL| 4.250921708424983|134896|
|          HARBOR| 3.702561599356521|126747|
|      HOLLENBECK|2.6801812377068184|115837|
|WEST LOS ANGELES| 2.792457289034123|115781|
|          NEWTON| 1.634635739709743|111110|
|       NORTHEAST| 3.623665524604087|108109|
|         

In [28]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 11.18 seconds

## Αποτελέσματα

| # Executors | Cores | Memory (GB) | Time (seconds) |
|---|---|---|---|
| 2 | 4 | 8 | 41.76 |
| 4 | 2 | 4 | 19.32 |
| 8 | 1 | 2 | 11.18 |